In [13]:
!pip install zemberek-python

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.1/95.1 MB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 112.4/112.4 kB 11.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for antlr4-python3-runtime: filename=antlr4_python3_runtime-4.8-py3-none-any.whl size=141210 sha256=69f748904231e1460e2ded62ed4999eec5acdc386a042e3f84fd5a341007c084
  Stored in directory: /root/.cache/pip/wheels/a7/20/bd/e1477d664f22d99989fd28ee1a43d6633dddb5cb9e801350d5
Successfully built antlr4-python3-runtime


In [1]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [46]:
def check_keyword_balance(text, up_keywords, down_keywords):
    up_count = 0
    down_count = 0
    up_positions = []
    down_positions = [] 
    words = text.lower().split()

    for i, word in enumerate(words):
        if word in up_keywords:
            up_count += 1
            up_positions.append(i+1) 

        if word in down_keywords:
            down_count += 1
            down_positions.append(i+1) 

    if up_positions and down_positions:
        if up_positions[-1] > down_positions[-1]:
            up_count = 1.2 * up_count
        else:
            down_count = 1.2 * down_count

    if up_count > down_count:
        return 1
    elif down_count > up_count:
        return 0
    else:
        if up_count == 0 and down_count == 0:
            return -1  # Alakasız metin
        else:
            return 2  # Nötr metin



In [48]:
from zemberek import TurkishMorphology
import os
import shutil

# Initialize the Turkish Morphology object
morphology = TurkishMorphology.create_with_defaults()


# Define the project path
project_path = "/content/drive/My Drive/Project_cs210/sorted_news/"

# Define subdirectories
subdirs = ["neg", "pos"]

# Initialize counters
count_neg = 0
count_pos = 0
count_neutral = 0
count_irrelevant = 0
moved_counter_stemmed = 0
moved_counter = 0

# Define your keyword lists
up_keywords = ['onay','kazandı', 'tamamlandı', 'zirve', 'zirve!','yüksek', 'aştı', 'şampiyon', 'ralli', 'ileri', 'kırpıyor','rekor','yukarı','yüksek' ,'yükseliş', 'toparlama', 'toparlanma', 'ılımlı', 'yükselişle','yukarı', 'alıcılı', 'iyimser', 'alım','pozitif','olumlu', 'güçlü', 'artan', 'arttı', 'alım', 'artı', 'pozitif', 'iştah', 'fırsat', 'güç', 'yükseldi', 'kar', 'bahar', 'rekor', 'aşma', 'üst', 'üzeri','yükselişte', 'hızlı', 'halka', 'zirvesinde', 'girişi', 'kazandırdı', 'rekoru!', 'rekorlar', 'rekoru', 'güçlenecek', 'açılıyor', 'primli', 'zirvesini', '\'ralli\'','artmaya', 'korunuyor', 'likit', 'anlaştı', 'tavan', 'rekorla', 'kazandırdı', 'şampiyonu', 'zirve:', 'doymuyor', 'zirvesi', 'rekorunu', 'coşturdu', 'artıyor', 'yükselişte','kazancında']
down_keywords = ['sakin', 'kaybında','geriledi' ,'kayıpla', 'ikilemi', 'düşüşle', 'geriliyor', 'deprem!','yatay','düşük', 'satış', 'düşüş', 'satıcılı', 'satışlar', 'uyarı','askı','dalgalı', 'kayıp', 'gerile', 'karışık','negatif' ,'azalan', 'baskılama', 'azalt', 'aşağı', 'sert', 'satış', 'geri', 'çekilme', 'eksi' , 'olumsuz', 'risk', 'düzeltme', 'baskı', 'dikkatli', 'temkinli', 'zayıf', 'kayıp', 'düştü', 'zarar', 'alt', 'kaçıyor']

# Stem the keyword lists
up_keywords_stemmed = []
down_keywords_stemmed = []

# Loop over the subdirectories
for subdir in subdirs:
    # Generate the full path to the subdir
    subdir_path = os.path.join(project_path, subdir)

    # Define target directory for mislabeled files
    target_dir = os.path.join(project_path, "pos" if subdir == "neg" else "neg")

    # List all files in the subdir
    file_list = os.listdir(subdir_path)
    
    # Loop over the files
    for file_name in file_list:
        # Generate the full path to the file
        file_path = os.path.join(subdir_path, file_name)
        
        # Open and read the file
        with open(file_path, 'r', encoding='iso-8859-9') as f:
            content = f.read()

        # Check keyword balance without stemming
        result = check_keyword_balance(content, up_keywords, down_keywords)


        # Check if file is mislabeled and move if necessary
        if (subdir == "neg" and result == 1) or (subdir == "pos" and (result == 0 or result == 2)):
            target_path = os.path.join(target_dir, file_name)
            shutil.move(file_path, target_path)
            print(f"Moved mislabeled file {file_name} to {target_dir}")  
            moved_counter += 1


        # Update counters
        if result == 0:
            count_neg += 1
        elif result == 1:
            count_pos += 1
        elif result == 2:
            count_neutral += 1
            os.remove(file_path)
        else:
            os.remove(file_path)
            count_irrelevant += 1

# Print results
print("Negative: ", count_neg)
print("Positive: ", count_pos)
print("Neutral: ", count_neutral)
print("Irrelevant: ", count_irrelevant)


INFO:zemberek.morphology.turkish_morphology:TurkishMorphology instance initialized in 17.888945817947388


2023-06-09 10:52:19,780 - zemberek.morphology.turkish_morphology - INFO
Msg: TurkishMorphology instance initialized in 17.888945817947388

Negative:  764
Positive:  942
Neutral:  5
Irrelevant:  74


In [49]:
print(moved_counter)
print(moved_counter_stemmed)


0
0


In [33]:
!pip install nltk

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


**didn't use below**

In [58]:
import nltk
from nltk.corpus import stopwords
# Download stopwords
nltk.download('stopwords')

# Get the list of Turkish stopwords
stopwords_turkish = stopwords.words('turkish')
# Define the project path
project_path = "/content/drive/My Drive/Project_cs210/sorted_news/"
processed_path = "/content/drive/My Drive/Project_cs210/processed_files/"
# Define subdirectories
subdirs = ["neg", "pos"]
# Loop over the subdirectories
for subdir in subdirs:
    # (The rest of your loop goes here)
    # Generate the full path to the subdir
    subdir_path = os.path.join(project_path, subdir)

    # Define target directory for mislabeled files
    target_dir = os.path.join(project_path, "pos" if subdir == "neg" else "neg")

    # List all files in the subdir
    file_list = os.listdir(subdir_path)
    # Loop over the files
    for file_name in file_list:
        # Generate the full path to the file
      file_path = os.path.join(subdir_path, file_name)
      # Open and read the file
      with open(file_path, 'r', encoding='iso-8859-9') as f:
          content = f.read()
      
      # Remove stopwords
      words = content.lower().split()
      words = [word for word in words if word not in stopwords_turkish]
      content = " ".join(words)

      # Write processed text to new file in processed_files directory
      new_subdir_path = os.path.join(processed_path, subdir)
      os.makedirs(new_subdir_path, exist_ok=True)
      new_file_path = os.path.join(new_subdir_path, file_name)
      with open(new_file_path, 'w', encoding='utf-8') as f:
          f.write(content)
          print(content)
          print("executed")

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


bist-100 dün yatay kapanış gerçekleştirdi. hafta başından yana olumsuz yönde ayrışan banka hisseleri endeksin tutunmasına destek verirken, ayrıca otomotiv perakende hisselerinde olumlu ayrışma gözledik. buna karşılık geride kalan alt sektörlerin pek çoğunda satış baskısı devam etti. bilanço beklentilerine bağlı hisse bazlı hareketler bist-100deki sıkışık seyrin bir süre devamını bekliyoruz.
executed
piyasa dün beklentiler paralelinde sert bir düzeltme gerçekleştirdi pek hissenin taban fiyat seviyelerinden alıcılı bulamadığı bir seansın ardından bist-100 endeksi günü %10 kayıpla tamamladı. yurt dışı tarafta abd tahvil faizlerinde hafif bir gerileme abd hisselerinde ılımlı bir yükseliş gözlendi. sabah işlemlerinde yurt dışı hisse senetleri tarafı nispeten sakin, yurt içinde kur faiz seviyeleri dünkü kapanışlarına yakın görünüyor. dünkü sert düşüşün ardından bistde dengelenme çabasının bugün sürmesini endeksin güne bir miktar satış baskısı başlamasını bekliyoruz.
executed
geçtiğimiz haf

In [ ]:

# Loop over the subdirectories
for subdir in subdirs:
    # Generate the full path to the subdir
    subdir_path = os.path.join(project_path, subdir)

    # Define target directory for mislabeled files
    target_dir = os.path.join(project_path, "pos" if subdir == "neg" else "neg")

    # List all files in the subdir
    file_list = os.listdir(subdir_path)
    
    # Loop over the files
    for file_name in file_list:
        # Generate the full path to the file
        file_path = os.path.join(subdir_path, file_name)
        
        # Open and read the file
        with open(file_path, 'r', encoding='iso-8859-9') as f:
            content = f.read()

        # Check keyword balance without stemming
        result = check_keyword_balance(content, up_keywords, down_keywords)


        # Check if file is mislabeled and move if necessary
        if (subdir == "neg" and result == 1) or (subdir == "pos" and (result == 0 or result == 2)):
            target_path = os.path.join(target_dir, file_name)
            shutil.move(file_path, target_path)
            print(f"Moved mislabeled file {file_name} to {target_dir}")  
            moved_counter += 1


        # Update counters
        if result == 0:
            count_neg += 1
        elif result == 1:
            count_pos += 1
        elif result == 2:
            count_neutral += 1
            os.remove(file_path)
        else:
            os.remove(file_path)
            count_irrelevant += 1
